## Logistic Regression

**Logistic Regression** is a simple and interpretable model used for **binary classification**.

### Formula
- Linear output:  
  $$z = \mathbf{w}^\top \mathbf{x} + b$$
- Sigmoid activation:  
  $$\hat{y} = \sigma(z) = \frac{1}{1 + e^{-z}}$$

### Loss Function
- **Binary Cross-Entropy (BCE)**:  
  $$\mathcal{L}(y, \hat{y}) = -y \log(\hat{y}) - (1 - y) \log(1 - \hat{y}) $$

### Characteristics
- ✅ Fast and easy to implement  
- ✅ Probabilistic output  
- ❌ Limited to linear decision boundaries

### Common Use Case
- Email spam detection  
- Medical diagnosis (e.g., diabetes prediction)


In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
import numpy as np

def My_sigmoid(z):
    return 1 / (1 + torch.exp(-z))

def My_binary_cross_entropy(pred, target):
    epsilon = 1e-7  # Avoid log(0)
    pred = torch.clamp(pred, epsilon, 1 - epsilon)
    return -(target * torch.log(pred) + (1 - target) * torch.log(1 - pred)).mean()  

# ===== Logistic Regression Model =====
class LogisticRegression(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, 1)

    def forward(self, x):
        return My_sigmoid(self.linear(x)) # or use torch.sigmoid(self.linear(x))

# ===== Only keep 3 and 8 =====
def filter_3_and_8(dataset):
    idx = (dataset.targets == 3) | (dataset.targets == 8)
    dataset.targets = dataset.targets[idx]
    dataset.data = dataset.data[idx]
    # 3 -> 0, 8 -> 1
    dataset.targets = (dataset.targets == 8).long()
    return dataset

def main():
    batch_size = 64
    lr = 0.1
    num_epochs = 10

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.view(-1))  # Flatten 784-dimensional vector, or use nn.Flatten()
    ])

    train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

    train_dataset = filter_3_and_8(train_dataset)
    test_dataset = filter_3_and_8(test_dataset)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=1000)

    model = LogisticRegression(784)
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)

    # ===== Training =====
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0
        for x, y in train_loader:
            pred = model(x).squeeze(1)
            criterion = nn.BCELoss()  # or use loss = My_binary_cross_entropy(pred, y.float())
            loss = criterion(pred, y.float())  
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * x.size(0)
            preds = (pred > 0.5).long()
            correct += (preds == y).sum().item()
            total += y.size(0)

        acc = correct / total
        print(f"Epoch {epoch+1}, Loss: {total_loss/total:.4f}, Accuracy: {acc:.4f}")

    # ===== Testing =====
    model.eval()
    with torch.no_grad():
        for x, y in test_loader:
            pred = model(x).squeeze(1)
            preds = (pred > 0.5).long()
            acc = (preds == y).float().mean().item()
            print(f"Test Accuracy: {acc:.4f}")
            break

if __name__ == "__main__":
    main()


Epoch 1, Loss: 0.2065, Accuracy: 0.9350
Epoch 2, Loss: 0.1347, Accuracy: 0.9583
Epoch 3, Loss: 0.1220, Accuracy: 0.9621
Epoch 4, Loss: 0.1157, Accuracy: 0.9639
Epoch 5, Loss: 0.1121, Accuracy: 0.9649
Epoch 6, Loss: 0.1085, Accuracy: 0.9664
Epoch 7, Loss: 0.1067, Accuracy: 0.9676
Epoch 8, Loss: 0.1052, Accuracy: 0.9680
Epoch 9, Loss: 0.1038, Accuracy: 0.9691
Epoch 10, Loss: 0.1022, Accuracy: 0.9695
Test Accuracy: 0.9550
